**TMDB API (Practice)** 

This practice assignment will reinforce important learning objectives from the previous lesson(s), and allow you to take on more challenging core assignments, preparing you for graduation.

Practice and tinker with this assignment until you're comfortable performing each of the tasks. Then, be sure to submit your output as described in the steps below.

**TMDB API (Practice):**

**Project Planning**

As discussed in the previous lesson, for the next part of your project, you will extract financial and certification data from TMDB's API for your IMDB data set. You will use an OUTER and INNER loop: a loop within a loop!

The OUTER loop will loop through the start years included in the IMDB data, filter the title basics data for the selected year, and save the list of movie ids from that year to retrieve in the inner loop.

The INNER loop loops through every movie id from the selected year, extracts its results from the TMDB API, and appends them to a JSON file.

# **For this practice assignment**

You will be practicing the inner loop of API calls for a single year's list of movies from your IMDB title basics data. Specifically, you will extract the API results for every movie with a startYear of 2000.

* **Read the instructions below, including the examples in the "Getting Started" section, before starting your work.**

* **Create a new notebook in your project repository called "Practicing TMDB API calls.**

**Preparation BEFORE the loop**
* Designate a folder to save your information.
* Define custom functions you will use for your API calls
* Load your cleaned title basics data from Part 1 of Project 2 (or query your title_basics table in your MySQL database).
* Define the year you wish to retrieve (2010) and create an empty list for appending error messages.

**Prepare the DataFrame and JSON File**
* **Use the selected year to define filenames and filter the data**
    1. Define a JSON_FILE filename to save the results in progress.
    2. Check if the file exists.
        * if it does not exist, create the empty JSON file with with open that just contains the key "imdb_id"
        * if it exists, print a message saying that it already exists.

***Now that the JSON file for the results in progress exists:***
* Filter the IMDB title basics data for the selected year and save the movie IDs from that year as "movies_ids".
* Check the JSON file for previously downloaded movie IDs and filter out the movie ids that already exists in the JSON file ( to prevent duplicate API calls) by:
    * Loading in the contents of the JSON file pd.read_json.
        * Compare the movie_ids that were in the JSON file to your saved movie_ids_to_get.
    * Save the final list of "movie_ids_to_get" by filtering out movies that already exists in the JSON file.

**Perform the Loop of API Calls**

Note: you have already written a function to combine the certification with the rest of the .info() from the TMDB API results in the Intro to TMDB API lesson.

**Create a loop to make API calls for each id** in the YEAR specified. Include a progress bar using tqdm_notebook.

***For each movie id:***
* Extract the current ID from the API and retrieve the dictionary of results
* Append the new results to the list from the JSON file
* Save the updated JSON file back to the disk

**Save the Results to Compressed .csv**
* **After the loop**, save the final results for the year as a csv.gz file with the year in the filename.

Note: at this point, you'll have completed the inner loop that you will need for the next part of the project.

# **Getting Started**

## **Preparation BEFORE the Loop:**

**Install, Import packages**

In [1]:
pip install tmdbsimple

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import packages
import os, time, json
import pandas as pd

In [3]:
from tqdm.notebook import tqdm_notebook

In [4]:
import tmdbsimple as tmdb

In [5]:
# Load TMDB API Key and add to tmdbsimple
with open(r'C:\Users\ASUS TUF\Documents\GitHub\ods-pt-data-enrichment-project\.secret\tmdb_api.json') as f:
    login = json.load(f)
print(login.keys())

dict_keys(['api-key'])


In [6]:
# Importing tmdbsimple and setting the API_KEY

import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

**Designate a folder**

You will save API call data in the data folder you created for project Part 1.

In [7]:
# Create the folder for saving files (if it doesn't exist)
FOLDER = "PracticeData/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'title-akas-us-only.csv',
 'title-basics-us-only.csv',
 'title-ratings-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'tmdb_api_results_2000.json']

In [8]:
# If you created the data folder for part 1, you will see your csv files listed here. If not, it will just be empty [].

**Define Your Functions**

You should ultimately put any custom functions at the top of your notebook. You can first write them where you first use them in your project, but once you have the functions completed and tested, you should move their definitions to the top of your notebook after you import your packages.

You will need your function to get the movie rating from the prior lesson, as well as the new function below: write_json. This is a modified version of a function from [https://www.geeksforgeeks.org/append-to-json-file-using-python/](https://www.geeksforgeeks.org/append-to-json-file-using-python/). Notice that the original source link is included in the function's docstring to give proper credit to the original authors.

In [9]:
# Define your functions
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info


def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

**Confirm Your API Function works!**

In order to ensure your function for extracting movie data from TMDB is working, test your function on these 2 movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook"). Make sure that your function runs without error and that it returns the correct movie's data for both test ids.

**Load in the Cleaned Title Basics data**

You need to read in the filtered dataframe you created based on the specification of Project 2 Part 1.

You will be filtering out the movies for each year inside the loop, so we will need this loaded and ready to be filtered.

In [10]:
# Load the cleaned Title Basics (from Part 1)
basics = pd.read_csv('Data/title-basics-us-only.csv')
basics

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,67666,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,86793,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,93930,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...,...
86974,10016149,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
86975,10016544,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
86976,10016684,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
86977,10016693,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


**Define a variable with the year to Extract from the API**

We have data from 2000 - 2020 available. For this assignment, you will just retrieve data for the year 2010. We will save the year as a variable so that we can easily reuse it within our code and also our filenames.

**Define an errors list**

We will want to be able to save the ids and error messages for any movie that causes an error. To do so, we will want to create an empty errors list before our loops that we can append to later.

In [11]:
# Set the year to filter for
YEAR = 2010

# Create an empty list for saving errors
errors = []

**Prepare the DataFrame and JSON File**

* Select a JSON_FILE filename to save the results in progress.

First, define the file path, including the year. For the project, you are going to have multiple files, one for each year of movies. The code below will identify the folder in the FOLDER we just defined above and will name the file based on the current year.

Determine if the JSON file exists:

Check if that file already exists or not. If you are going through this lesson for the first time, it is very unlikely that the file exists! But, if you are at a different point in the project, and it already exists, we don't need to do anything, but just make sure it is a file you want to add to!

If it does not exist:

* Print f"Creating {filename} for API results for {YEAR}."
* Create an empty JSON file using with open that just contains a dictionary with the key "imdb_id" and the value 0.
    * We will be appending to this empty dictionary throughout our calls.
* If it already exists:
    * Print "The file {JSON_FILE} already exists."

In [12]:
# Define the JSON file to store results for the year
JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'


# Check if the JSON file exists
file_exists = os.path.isfile(JSON_FILE)

# If it does not exist: create it
if file_exists == False:
    print(f"Creating {JSON_FILE} for API results for year={YEAR}.")
    
    # save an empty dict with just "imdb_id" to the new json file.
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)

# If it exists, print a message
else:
    print(f'The file {JSON_FILE} already exists.')

Creating Data/tmdb_api_results_2010.json for API results for year=2010.


* Filter for the selected year and save the movie ids**

For the project, you will be breaking up the title_basics data by year. For this practice assignment, we will only be extracted data for the year 2010.

We will create a new DataFrame by filtering title_basics for the selected YEAR. We will then save the list of movie_ids as a separate variable.

In [13]:
# Filtering for movies from selected startYear
df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
movie_ids = df['tconst']
movie_ids.head()

1138    tt0230212
3806    tt0312305
4258    tt0326965
4436    tt0331312
6670    tt0393049
Name: tconst, dtype: object

**Check previous results and create the final list of movie_ids_to_get**

You may remember from our lesson on efficient API calls that we are going to build in some safeguards when looping through multiple calls.

    * Load in any existing API results with pd.read_json
    * Check to see if any of the movie_ids to get are already in the JSON file.
    * Filter out only movies that are missing from the JSON file to use in the loop

The code loads any existing information from the JSON file into a dataframe called the "previous_df." This will start empty, but as you iterate through the loop, it will continue to have more and more information.

In [14]:
# Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)
previous_df

,imdb_id
0,0


**Check for and filter out movie IDs that already exist**

The next line of code will prevent you from wasting API calls on data you already have. Note that it is defining the ids you are calling in such a way that it excludes any ids that are already present in the previous_df. You may recall that this will also allow you to "pick up where you left off" if your API call gets interrupted.

In [15]:
# filter out any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

Now we have defined the "movie_ids_to_get". It includes the ids from our dataframe in the year we are seeking, and it excludes any that we have already made calls for. We will use this list for our loop of API calls.

**Start Loop Through Movie IDs**

Now that we have the filtered list of movie_ids_to_get for the current year, we will now create an inner loop to iterate through the movie_ids_to_get, and for each ID, we will: retrieve the movie info from the TMDB API, append the movie_info dictionary to our JSON_FILE, wait 20 ms to avoid overwhelming the API.

**Set up Progress Bar*

We want to keep track of our progress and ensure our calls are working. The progress bar works within the for statement of the for loop. Note that this will iterate through each year that is defined in the YEARS_TO_GET variable.

In [16]:
# Loop through movie_ids_to_get with a tqdm progress bar
for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}"):

    # Attempt to retrieve then data for the movie id
    try:
        temp = get_movie_with_rating(movie_id)  #This uses your pre-ma    de function
        # Append/extend results to existing file using a pre-made function
        write_json(temp,JSON_FILE)
        # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)

    # If it fails,  make a dict with just the id and None for certification.
    except Exception as e:
        errors.append([movie_id, e])

Movies from 2010:   0%|          | 0/3862 [00:00<?, ?it/s]

***Earlier, ran year 2000***

Ultimately we will be creating a loop, but let's explore each piece of the code:

**Iterate through the list of Movie IDs and make the calls**

The code below relies on the function you wrote in the previous lesson that made API calls and added the certification to the .info results. Here this function is named "get_movie_with_rating". Make sure you have the function from the earlier lesson in the code file before you plan to call on it! This loop also uses the function above (write_json) to extend/append the results to the .json file. **Make sure both functions are defined in your code file before you try to call them!**

Since some movies exist in IMDB's title basics dataset (our DataFrame) that do not exist within the database for TMDB's API, we will get an error whenever we attempt to retrieve a movie id that TMDB does not have in its database.

To get around this, we will use a try and except statement around our API extraction code. We will TRY to retrieve and save the data for the current movie_id, but if we get an error, we will save the movie_id and error message in our errors list.

**After the Loop**

*Print a message reporting back the number of movie ids that caused an error.

In [17]:
print(f"- Total errors: {len(errors)}")

- Total errors: 1114


Once the inner loop through the movie_ids_to_get has finished, we will have all of our results for that year in our JSON_FILE. We now want to save them in a smaller file format.

**Save the year's results as csv.gz file**

Once all of the API calls for the current year are made, you should open your .json file with pd.read_json and convert each json file to a compressed csv (".csv.gz") to save space. This is done after the loop.

In [18]:
# Save the final results to a csv.gz file
final_year_df = pd.read_json(JSON_FILE)

csv_fname = f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"
final_year_df.to_csv(csv_fname, compression="gzip", index=False)

# Troubleshooting:

If you get an error message when trying to run pd.read_json, try replacing pd.read_json with the "read_and_fix_json" helper function in this repository: https://github.com/coding-dojo-data-science/data-enrichment-helper-functions

# Summary

This lesson exemplifies the importance of planning your complex coding tasks so that you are clear on what you are trying to do in plain language before translating to code. While this lesson shows examples of the some of the code that you may want to use in the next phase of the project, remember it is still up to you to read and understand each step so that you can put together the final product!